### Import all necessary libraries

In [132]:
import pyvista as pv
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,root_mean_squared_error,r2_score,mean_absolute_error,accuracy_score
import numpy as np

### Define the function for reading any mesh data

In [133]:
def _get_foam_file(file_path):
    for file in os.listdir(file_path):
        if file.endswith('.foam'):
            return os.path.join(file_path, file)
        
def _read_mesh_data(work_dir):

    reader = pv.OpenFOAMReader(_get_foam_file(work_dir))
    last_time_step = int(reader.time_values[-1])
    reader.set_active_time_value(last_time_step)
    # reader.cell_to_point_creation = False

    internal_mesh = reader.read()["internalMesh"]
    boundaries = reader.read()["boundary"]

    internal_mesh.work_dir = work_dir
    internal_mesh.active_time_value = last_time_step

    return internal_mesh, boundaries

def _select_only_relevant_field_data(pvObject, fields):

    pvCopy = pvObject.copy()
    pvCopy.clear_data()
    for field in fields:
        if field in pvObject.array_names:
            pvCopy[field] = pvObject[field]
    
    pvCopy = pvCopy.cell_data_to_point_data() #converting cell data to point data
    return pvCopy

fields = ['epsilon', 'U', 'mag(U)','k', 'strainRate']

### Read the largest geometry mesh

In [134]:
largest_mesh_dir=r"D:\GPOD ANN Sartorius BioStat 50L 30 degrees removed geoms modified v3\Sartorius BioStat 50L\BIOSTAT_50L_Run_41"
largest_mesh,_=_read_mesh_data(largest_mesh_dir)
largest_mesh.clear_data()
largest_mesh

UnstructuredGrid,Information
N Cells,1004477
N Points,1185213
X Bounds,"-1.850e-01, 1.850e-01"
Y Bounds,"-1.850e-01, 1.850e-01"
Z Bounds,"-9.250e-02, 4.039e-01"
N Arrays,0


### Read the target mesh that is to be interpolated

In [135]:
target_mesh_dir=r"D:\GPOD ANN Sartorius BioStat 50L 30 degrees removed geoms modified v3\Sartorius BioStat 50L\BIOSTAT_50L_Run_07"
internal_mesh,_=_read_mesh_data(target_mesh_dir)
target_mesh=_select_only_relevant_field_data(internal_mesh, fields)
target_mesh

UnstructuredGrid (0x216e1f78160)
  N Cells:	832347
  N Points:	977284
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 2.359e-01
  N Arrays:	5

### Interpolate the target mesh against largest geometry with radius=0.001

In [99]:
interp_mesh = largest_mesh.interpolate(target_mesh, radius = 1e-3, progress_bar = True)

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:00<00:00]


In [100]:
interp_mesh = interp_mesh.point_data_to_cell_data()
interp_mesh

UnstructuredGrid (0x216aa47f2e0)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [113]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Interpolate the target mesh against largest geometry with radius=0.0155

In [199]:
bounds = target_mesh.extract_cells(1).bounds
radius = min([bounds[i+1]-bounds[i] for i in [0,2,4]])
print(radius*5)

0.015572737902402878


In [200]:
interp_mesh_v2 = largest_mesh.interpolate(target_mesh, radius = radius*5, progress_bar = True)
interp_mesh_v2 = interp_mesh_v2.point_data_to_cell_data()

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[01:24<00:00]


In [201]:
interp_mesh_v2

UnstructuredGrid (0x216deec1280)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [114]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Interpolate the target mesh against largest geometry with radius=0.0311

In [15]:
print(radius*10)

0.031145475804805756


In [16]:
interp_mesh_v3 = largest_mesh.interpolate(target_mesh, radius = radius*10, progress_bar = True)
interp_mesh_v3 = interp_mesh_v3.point_data_to_cell_data()

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[06:33<00:00]


In [57]:
interp_mesh_v3

UnstructuredGrid (0x20271196dc0)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [115]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v3.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Interpolate the target mesh against largest geometry with radius=0.05

In [40]:
interp_mesh_v4 = largest_mesh.interpolate(target_mesh, radius = 0.05, progress_bar = True)
interp_mesh_v4 = interp_mesh_v4.point_data_to_cell_data()

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[22:05<00:00]


In [ ]:
interp_mesh_v4

In [116]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v4.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Interpolate the target mesh against largest geometry with radius=0.008

In [117]:
interp_mesh_v5 = largest_mesh.interpolate(target_mesh, radius = 0.008, progress_bar = True)
interp_mesh_v5

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:15<00:00]


UnstructuredGrid (0x216b867c520)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [118]:
interp_mesh_v5 = interp_mesh_v5.point_data_to_cell_data()

In [121]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v5.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Interpolate the target mesh against largest geometry with radius=0.0015

In [192]:
interp_mesh_v6 = largest_mesh.interpolate(target_mesh, radius = 0.0015, progress_bar = True)
interp_mesh_v6=interp_mesh_v6.point_data_to_cell_data()
interp_mesh_v6

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:00<00:00]


UnstructuredGrid (0x216f03f0f40)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [193]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v6.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Interpolate the target mesh against largest geometry with radius=0.0075

In [180]:
interp_mesh_v7 = largest_mesh.interpolate(target_mesh, radius = 0.0075, progress_bar = True)
interp_mesh_v7=interp_mesh_v7.point_data_to_cell_data()
interp_mesh_v7

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:12<00:00]


UnstructuredGrid (0x216e2182880)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [181]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v7.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Interpolate the target mesh against largest geometry with radius=0.0071

In [190]:
interp_mesh_v8 = largest_mesh.interpolate(target_mesh, radius = 0.0073, progress_bar = True)
interp_mesh_v8=interp_mesh_v8.point_data_to_cell_data()
interp_mesh_v8

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:12<00:00]


UnstructuredGrid (0x216efa28040)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [191]:
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v8.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Code snippet to calculate regression metrics

In [70]:
#Store all the regression metrics in tabular format
def calc_regression_metrics(target_mesh,interp_mesh,true_idx,radius):
    
    df_metrics=pd.DataFrame(index=["R2_score","MSE","RMSE","MAE"])
    
    for field in fields:
        if field == 'U':
            for i in range(3):
                true_values=np.round(target_mesh[f"{field}"][:,i],4)
                pred_values=np.round(interp_mesh[f"{field}"][true_idx][:,i],4)

                r2_score_value=r2_score(true_values,pred_values)
                mse_value=mean_squared_error(true_values,pred_values)
                rmse_value=root_mean_squared_error(true_values,pred_values)
                mae_value=mean_absolute_error(true_values,pred_values)

                df_metrics[f"{field}_{i}"]=[r2_score_value,mse_value,rmse_value,mae_value]

        true_values=np.round(target_mesh[f"{field}"],4)
        #print(true_values.shape)
        pred_values=np.round(interp_mesh[f"{field}"][true_idx],4)
        #print(pred_values.shape)
                
        r2_score_value=r2_score(true_values,pred_values)
        mse_value=mean_squared_error(true_values,pred_values)
        rmse_value=root_mean_squared_error(true_values,pred_values)
        mae_value=mean_absolute_error(true_values,pred_values)
        
        df_metrics[f"{field}"]=[r2_score_value,mse_value,rmse_value,mae_value]
        
        df_metrics.to_excel(f"Regression_mterics_with_radius_{radius}.xlsx")

In [71]:
target_mesh_copy=target_mesh.copy()
target_mesh_copy=target_mesh_copy.point_data_to_cell_data()
target_mesh_copy.cell_centers().points.shape

(832347, 3)

In [148]:
idx=interp_mesh.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.001

calc_regression_metrics(target_mesh_copy,interp_mesh,idx,f"{radius}")

(832347,)


In [152]:
mismatch_indices=np.where(interp_mesh["mag(U)"][idx]!=target_mesh_copy["mag(U)"])
mismatch_indices[0]

array([    71,    127,    128, ..., 832339, 832340, 832341], dtype=int64)

In [153]:
target_mesh_copy["mag(U)"][mismatch_indices[0]]

pyvista_ndarray([0.21905002, 0.22270617, 0.22270711, ..., 0.4389211 ,
                 0.42003384, 0.42939174], dtype=float32)

In [155]:
interp_mesh["mag(U)"][mismatch_indices[0]]

pyvista_ndarray([0.25954586, 0.23841754, 0.25752276, ..., 0.2346508 ,
                 0.22842461, 0.22580338], dtype=float32)

In [77]:
idx=interp_mesh_v2.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.0155

calc_regression_metrics(target_mesh_copy,interp_mesh_v2,idx,f"{radius}")

(832347,)


In [78]:
idx=interp_mesh_v3.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.0311

calc_regression_metrics(target_mesh_copy,interp_mesh_v3,idx,f"{radius}")

(832347,)


In [79]:
idx=interp_mesh_v4.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.05

calc_regression_metrics(target_mesh_copy,interp_mesh_v4,idx,f"{radius}")

(832347,)


In [119]:
idx=interp_mesh_v5.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.008

calc_regression_metrics(target_mesh_copy,interp_mesh_v5,idx,f"{radius}")

(832347,)


In [143]:
idx=interp_mesh_v6.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.0015

calc_regression_metrics(target_mesh_copy,interp_mesh_v6,idx,f"{radius}")

(832347,)


In [183]:
idx=interp_mesh_v7.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.0075

calc_regression_metrics(target_mesh_copy,interp_mesh_v7,idx,f"{radius}")

(832347,)


In [194]:
# Visualizing how true mesh looks like
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(target_mesh_copy.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [203]:
# Superimpose the interpolated points at idx
import time

start=time.time()
idx_v2=interp_mesh_v2.find_containing_cell(target_mesh_copy.cell_centers().points)
true_idx=target_mesh_copy.find_containing_cell(target_mesh_copy.cell_centers().points)

interp_mesh_v2_copy=interp_mesh_v2.copy()
interp_mesh_v2_copy.clear_data()

for field in ['strainRate', 'epsilon', 'k', 'U', 'mag(U)']:
    #Get the flow field data of both meshes
    interp_field=interp_mesh_v2[field]
    true_field=target_mesh_copy[field]
    
    interp_field[idx_v2]=true_field[true_idx]
    
    interp_mesh_v2_copy[field]=interp_field
    
end=time.time()
print(end-start)

28.017292022705078


In [204]:
# Visualizing how true mesh looks like
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v2_copy.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [130]:
target_mesh_copy=target_mesh.copy()
target_mesh_copy=target_mesh_copy.point_data_to_cell_data()
target_mesh_copy.cell_centers().points.shape

(832347, 3)

In [140]:
target_mesh_copy.cell_centers().points

pyvista_ndarray([[-0.12123248,  0.06537949, -0.0618345 ],
                 [-0.11858802,  0.06511667, -0.06201188],
                 [-0.12141301,  0.06234978, -0.0621354 ],
                 ...,
                 [ 0.00842871, -0.00330752,  0.21588929],
                 [ 0.0056474 , -0.00188564,  0.21593234],
                 [ 0.00706866, -0.00196502,  0.2158997 ]])

In [123]:
idx=interp_mesh.find_containing_cell(target_mesh_copy.cell_centers().points)

In [124]:
idx

array([ 63788,  63755,  56533, ..., 856996, 872414, 872418])

In [125]:
interp_mesh_v5.cell_centers().points[idx]

pyvista_ndarray([[-0.12123248,  0.06537949, -0.06183436],
                 [-0.11858785,  0.06511658, -0.0620116 ],
                 [-0.12141301,  0.06234973, -0.06213525],
                 ...,
                 [ 0.00842869, -0.00330725,  0.21588931],
                 [ 0.00564739, -0.00188563,  0.21593236],
                 [ 0.00706863, -0.00196496,  0.21589972]])

In [126]:
idx.shape

(832347,)

In [136]:
interp_values=interp_mesh["strainRate"][idx]
interp_values.shape

(832347,)

In [137]:
interp_values

pyvista_ndarray([15.804449, 14.031167, 17.393091, ..., 12.605095,
                 14.106081, 13.348893], dtype=float32)

In [138]:
target_mesh_copy["strainRate"]

pyvista_ndarray([15.804449, 14.031167, 17.393091, ..., 12.605095,
                 14.106081, 13.348893], dtype=float32)

In [ ]:
target_mesh_copy["strainRate"]

In [139]:
true_values= target_mesh_copy["strainRate"]
pred_values=interp_mesh["strainRate"][idx]

r2_score_value=r2_score(true_values,pred_values)
mse_value=mean_squared_error(true_values,pred_values)
rmse_value=root_mean_squared_error(true_values,pred_values)
mae_value=mean_absolute_error(true_values,pred_values)
(r2_score_value,mse_value,rmse_value,mae_value)

(0.9979174334607773, 2.307113, 1.5189184, 0.12630153)